In [1]:
from db.db import backup_database

backup_destination = r"N:\PPDO\BS\Records Department\Archive_Data"
backup_database(backup_dir=backup_destination, compress=True)

'N:\\PPDO\\BS\\Records Department\\Archive_Data\\archives_backup_20250905_103845.sql.bz2'

In [ ]:
from sqlalchemy.orm import Session
from sqlalchemy import or_
import pandas as pd
import numpy as np
from db import get_db_engine
from db.models import FileLocation, File, FileEmbedding
from text_extraction.pdf_extraction import PDFTextExtractor
from embedding.minilm import MiniLMEmbedder
from utils import extract_server_dirs, build_file_path
server_mount = r"N:\PPDO\Records"
target_filepath = r"N:\PPDO\Records\23xx   Crown College (College 3)\2318\2318\H - Submittals\Lumber_ProductData.pdf"
target_location = r"N:\PPDO\Records\37xx   Oakes College\3701\3701"


pdf_extractor = PDFTextExtractor()
embedder = MiniLMEmbedder()

text = pdf_extractor(target_filepath)
embeddings = embedder.encode(text)
target_loc_dirs = extract_server_dirs(full_path=target_location, base_mount=server_mount)
engine = get_db_engine()

target_sim_distance = lambda file_embeds: 1 - np.dot(file_embeds, embeddings)
build_local_path = lambda row: build_file_path(base_mount=server_mount, server_dir=row['file_server_directories'], filename=row['filename'])

with Session(engine) as db_session:
    files_located_in_dir = or_(
        FileLocation.file_server_directories == target_loc_dirs,
        FileLocation.file_server_directories.startswith(target_loc_dirs + '/')
    )

    query = db_session.query(FileLocation, FileEmbedding)\
        .join(File, FileLocation.file)\
        .join(FileEmbedding, File.hash == FileEmbedding.file_hash)\
        .filter(files_located_in_dir)
    
    # query into dataframe
    df = pd.read_sql(query.statement, db_session.bind)
    df['similarity'] = df['minilm_emb'].apply(target_sim_distance)
    df = df.sort_values(by='similarity', ascending=True)

    # Add local file path
    df['local_filepath'] = df.apply(build_local_path, axis=1)

df.head()




2025-09-10 16:05:29,481 db.db INFO Creating database engine


,id,file_id,existence_confirmed,hash_confirmed,file_server_directories,filename
0,1467780,742150,2025-08-15 23:46:32.559459,2025-08-05 01:37:35.595464,83xx Physical Education Facility/8300/8300-0...,1. Hot Work Permit Exhibit H.pdf
1,1479620,680440,2025-07-18 22:50:48.333338,2025-07-05 02:41:27.475147,16xx Cowell College/1600/1600-120/G - Constr...,Hot Work Permit 3.21.11.pdf
2,1037221,701522,2025-08-01 19:04:01.664393,2025-07-06 03:15:38.372858,16xx Cowell College/1649/1649/G - Constructi...,File 1649 Hot Work Permit Cowell Commons-Dinin...
3,1309205,876522,2025-08-02 23:40:38.666248,2025-07-12 02:52:15.745664,27xx Applied Sciences Baskin Engineering/274...,File 2744 Hot Work Permit.pdf
4,1488237,685017,2025-08-05 15:50:34.225005,2025-07-29 03:26:38.035454,90xx Campus Utilities/9000/9000-030/G - Const...,Hot Work Permit 8.18.11.pdf
